In [ ]:
"""
@author: Lars Kistner
Affiliation: Department of Measurement and Control
Mechanical Engineering Faculty - Universty of Kassel
email: lars.kistner@mrt.uni-kassel.de
"""

import sys
sys.path.append("../")

import numpy as np
from numpy import nan
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

import lib

fig_width_row = 3.267 * 1.2

# Load training data

In [ ]:
data_train = pd.read_csv("data/wind_train.csv").to_numpy()

# polynominale regression 1. - 4. order zero and non zero

In [ ]:
# train data
x = np.rad2deg(data_train[:,0])
y = data_train[:,1]

# poly models
X_M04 = np.vstack([x**4, x**3, x**2, x, x**0-1]).T
X_M04_f = "$f_4 = a_4x^4 + a_3x^3 + a_2x^2 + a_1x$"
X_M03 = np.vstack([x**3, x**2, x, x**0-1]).T
X_M03_f = "$f_3 = a_3x^3 + a_2x^2 + a_1x$"
X_M02 = np.vstack([x**2, x, x**0-1]).T
X_M02_f = "$f_2 = a_2x^2 + a_1x$"
X_M01 = np.vstack([x, x**0-1]).T
X_M01_f = "$f_1 = a_1x$"

X_M4 = np.vstack([x**4, x**3, x**2, x, x**0]).T
X_M4_f = "$f_4 = a_4x^4 + a_3x^3 + a_2x^2 + a_1x + a_0$"
X_M3 = np.vstack([x**3, x**2, x, x**0]).T
X_M3_f = "$f_3 = a_3x^3 + a_2x^2 + a_1x + a_0$"
X_M2 = np.vstack([x**2, x, x**0]).T
X_M2_f = "$| \\hat{\\mathbf{v}}_\\mathrm{air} |$"
X_M1 = np.vstack([x, x**0]).T
X_M1_f = "$f_1 = a_1x + a_0$"

# pred
x_pred = np.linspace(0, 12, 100)

# Plots

In [ ]:
lib.use_locals(True, True, True)
plt.figure(figsize=[fig_width_row,fig_width_row*0.6], dpi=300)

# zero
models_list = [X_M2]
labels_list = [X_M2_f]
y_pred_list = []
params_list = []
x = np.rad2deg(data_train[:,0])
y = data_train[:,1]

plt.scatter(x, y, s=2, label="observation")
for i, X in enumerate(models_list):
    param = np.linalg.lstsq(X, y, rcond=None)[0]
    params_list.append(param)
    func = np.poly1d(param)
    print(param)
    print(lib.fehlermass(y, func(x)))
    y_pred = func(x_pred)
    y_pred_list.append(y_pred)
    plt.plot(x_pred, y_pred, "k", linewidth=2, label=labels_list[i])

plt.xticks(range(0,15,2))
plt.yticks(range(0,15,2))
plt.xlim([0,12])
plt.ylim([0,13])
plt.ylabel("$\\vert \\mathbf{v}_\\mathrm{air} \\vert$ in $\\mathrm{m\\;s^{-1}}$")
plt.xlabel("$\\phi$ in deg")
plt.legend()
plt.grid()

plt.tight_layout()
plt.savefig("../export/jsss_train_data.png")
plt.show()

# load validation dataset

In [ ]:
df = pd.read_csv("data/wind_vali.csv.zip", index_col=0)
print(df.shape)

# Validation

In [ ]:
def evaluate_poly_model(dataset, param):
    x = dataset.wind_t - np.min(dataset.wind_t)
    y = dataset.wind_len_xy

    orientation_res = dataset.att_len_xy
    p = np.poly1d(param)
    y_pred = p(np.rad2deg(orientation_res))
    return (x, y, y_pred)

for param_no, param in enumerate(params_list):
    lib.use_locals(True, True, True)
    fig = plt.figure(figsize=[fig_width_row, fig_width_row*1.2], dpi=300)
    for i, key in enumerate(["valiA", "valiB", "valiC"]):
        x, y, y_pred = evaluate_poly_model(df[df.dataset==key], param)
        y = lib.moving_average(y, 50)
        y_pred = lib.moving_average(y_pred, 50)
        diff = y - y_pred

        fig.add_subplot(3, 1, i+1)
        plt.yticks(range(0,10))

        offset = [10*10,130*10]

        if i == 0:
            plt.title("validation data A")
            plt.ylim([0,3])
        elif i == 1:
            plt.title("validation data B")
            plt.ylim([1,5])
        elif i == 2:
            plt.title("validation data C")
            plt.ylim([2,7])
            plt.xlabel("time in s")
        else:
            raise RuntimeError("This should not have happend")
        plt.ylabel("$\\vert \\mathbf{v}_\\mathrm{wind} \\vert$ in $\\mathrm{m\\;s^{-1}}$")
        
        plt.plot(x[offset[0]:offset[1]]-offset[0]/10, y[offset[0]:offset[1]], "b:", label="measured")
        plt.plot(x[offset[0]:offset[1]]-offset[0]/10, y_pred[offset[0]:offset[1]], "k-", label="estimated")
        plt.grid()
        plt.xlim([0, 120])
        if i == 0:
            #plt.legend(loc="lower right")
            plt.legend()
        print(f"{key}: {lib.fehlermass(y[offset[0]:offset[1]], y_pred[offset[0]:offset[1]])}")

    plt.tight_layout()
    plt.savefig(f"../export/jsss_wind_speed_vali_{param_no}.png")
    plt.show()